# 📊 Session 2b - Régression Multiple : Au-delà d'une Seule Variable

**Formation IA & ML - SupNum Nouakchott**  
**Formateur:** Mohamed Beydia - Vela Learning

---

## 🎯 Objectifs de cette Session

Dans la session précédente, nous avons appris la régression linéaire avec **une seule variable** (surface → prix). Maintenant, nous allons aller plus loin !

**🏠 Dans le monde réel, le prix d'une maison dépend de PLUSIEURS facteurs :**
- 📏 Surface (m²)
- 🛏️ Nombre de chambres
- 📅 Âge de la maison
- 📍 Localisation (quartier)
- 🚗 Nombre de places de parking
- 🏡 Type de maison (villa, appartement, etc.)

**🎯 Objectifs d'apprentissage :**
- [ ] Comprendre la **régression linéaire multiple**
- [ ] Apprendre à utiliser **plusieurs variables** en même temps
- [ ] Interpréter les **coefficients multiples**
- [ ] Gérer les **corrélations entre variables**
- [ ] Comparer les performances avec la régression simple

**💡 Question centrale :** Comment prédire le prix avec PLUSIEURS caractéristiques ?

---

## 🧠 Qu'est-ce que la Régression Multiple ?

### 🔄 Rappel : Régression Simple

**Dans la session précédente :**
```
Prix = w₀ + w₁ × Surface
```

- **1 seule variable explicative** : Surface
- **2 paramètres** : w₀ (intercept) + w₁ (pente)
- **Graphique** : Une droite dans un plan 2D

### 🚀 Régression Multiple

**Maintenant, avec plusieurs variables :**
```
Prix = w₀ + w₁×Surface + w₂×Chambres + w₃×Age + w₄×Quartier
```

- **Plusieurs variables explicatives** : Surface, Chambres, Age, Quartier
- **Plusieurs paramètres** : w₀, w₁, w₂, w₃, w₄
- **Graphique** : Un hyperplan dans un espace multidimensionnel

### 🤔 Pourquoi la Régression Multiple ?

**🎯 Avantages :**
- ✅ **Plus réaliste** : Prend en compte plusieurs facteurs
- ✅ **Plus précise** : Généralement de meilleures prédictions
- ✅ **Plus complète** : Comprend mieux les relations complexes
- ✅ **Contrôle des variables** : Peut isoler l'effet de chaque facteur

**⚠️ Challenges :**
- 🔸 **Plus complexe** : Plus difficile à visualiser et interpréter
- 🔸 **Multicolinéarité** : Quand les variables sont corrélées entre elles
- 🔸 **Overfitting** : Risque de trop coller aux données d'entraînement
- 🔸 **Plus de données** : Besoin de plus d'observations

### 💡 Analogie Simple

**🏠 Évaluation d'une maison :**
- **Régression simple** : "Le prix dépend seulement de la taille"
- **Régression multiple** : "Le prix dépend de la taille, mais aussi du quartier, de l'âge, du nombre de chambres, etc."

C'est évidemment plus réaliste !

## 🛠️ Setup - Préparation de l'Environnement

In [ ]:
# 📦 Imports essentiels pour la régression multiple
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
sns.set_palette("husl")

# Configuration pandas pour un meilleur affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✅ Environnement prêt pour la régression multiple !")
print("🏠 Nous allons maintenant prédire les prix avec PLUSIEURS variables !")

## 🏠 Création d'un Dataset Réaliste - Extension de Notre Exemple

Reprenons notre exemple des maisons, mais cette fois avec **plusieurs caractéristiques**.

In [ ]:
# 🏠 === CRÉATION D'UN DATASET MULTI-VARIABLES RÉALISTE ===

print("🏠 === EXTENSION DE NOTRE EXEMPLE AVEC PLUSIEURS VARIABLES ===")
print()
print("🎯 Objectif : Prédire le prix avec Surface + Chambres + Age + Quartier")
print("📝 Contexte : Nous sommes agents immobiliers avec des données plus complètes")
print()

# Créons des données réalistes avec plusieurs variables
np.random.seed(42)  # Pour la reproductibilité

# Générons 30 maisons avec des caractéristiques variées
n_maisons = 30

# Variables explicatives
surfaces = np.random.normal(80, 20, n_maisons)  # Surface: moyenne 80m², écart-type 20
surfaces = np.clip(surfaces, 40, 150)  # Entre 40 et 150 m²

chambres = np.random.choice([1, 2, 3, 4, 5], n_maisons, p=[0.1, 0.3, 0.35, 0.2, 0.05])

ages = np.random.randint(0, 50, n_maisons)  # Age entre 0 et 50 ans

# Quartier: 1=Centre (cher), 2=Résidentiel (moyen), 3=Périphérie (moins cher)
quartiers = np.random.choice([1, 2, 3], n_maisons, p=[0.2, 0.5, 0.3])
quartier_noms = {1: 'Centre', 2: 'Résidentiel', 3: 'Périphérie'}

# Création d'un prix réaliste basé sur TOUTES les variables
# Prix de base + effet de chaque variable
prix_base = 100  # Prix de base: 100k€
effet_surface = 1.8  # 1.8k€ par m²
effet_chambres = 15  # 15k€ par chambre
effet_age = -0.8  # -0.8k€ par année d'âge
effet_quartier = {1: 50, 2: 0, 3: -25}  # Centre: +50k€, Résidentiel: +0k€, Périphérie: -25k€

# Calcul du prix avec une petite composante aléatoire
prix = []
for i in range(n_maisons):
    prix_calcule = (prix_base + 
                   effet_surface * surfaces[i] + 
                   effet_chambres * chambres[i] + 
                   effet_age * ages[i] + 
                   effet_quartier[quartiers[i]] + 
                   np.random.normal(0, 15))  # Bruit aléatoire ±15k€
    prix.append(max(prix_calcule, 50))  # Prix minimum de 50k€

# Création du DataFrame
data = pd.DataFrame({
    'Surface_m2': np.round(surfaces, 1),
    'Chambres': chambres,
    'Age_annees': ages,
    'Quartier_code': quartiers,
    'Quartier_nom': [quartier_noms[q] for q in quartiers],
    'Prix_k€': np.round(prix, 1)
})

# Affichage des premières lignes
print("📊 === APERÇU DE NOTRE DATASET COMPLET ===")
print(data.head(10))
print()
print(f"📏 Taille du dataset : {len(data)} maisons avec {len(data.columns)-1} caractéristiques")
print()

# Statistiques descriptives
print("📈 === STATISTIQUES DESCRIPTIVES ===")
print(data.describe().round(2))
print()

# Comptons les quartiers
print("🏘️ === RÉPARTITION PAR QUARTIER ===")
quartier_stats = data.groupby('Quartier_nom')['Prix_k€'].agg(['count', 'mean', 'std']).round(1)
quartier_stats.columns = ['Nombre', 'Prix_Moyen_k€', 'Écart_Type_k€']
print(quartier_stats)
print()

print("💡 === OBSERVATIONS INITIALES ===")
print(f"   🏠 Prix moyen : {data['Prix_k€'].mean():.1f} k€")
print(f"   📏 Surface moyenne : {data['Surface_m2'].mean():.1f} m²")
print(f"   🛏️ Chambres en moyenne : {data['Chambres'].mean():.1f}")
print(f"   📅 Âge moyen : {data['Age_annees'].mean():.1f} ans")
print(f"   🎯 Notre objectif : Utiliser TOUTES ces variables pour prédire le prix !")

## 📊 Analyse Exploratoire - Relations entre Variables

Avant de construire notre modèle, analysons les relations entre nos variables.

In [ ]:
# 📊 === ANALYSE DES CORRÉLATIONS ===

print("📊 === ANALYSE DES CORRÉLATIONS ENTRE VARIABLES ===")
print()

# Créons une matrice de corrélation
variables_numeriques = ['Surface_m2', 'Chambres', 'Age_annees', 'Quartier_code', 'Prix_k€']
correlation_matrix = data[variables_numeriques].corr()

# Visualisation de la matrice de corrélation
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Masquer la partie supérieure
sns.heatmap(correlation_matrix, 
           annot=True, 
           cmap='RdBu_r', 
           center=0,
           mask=mask,
           square=True,
           fmt='.2f',
           cbar_kws={'shrink': 0.8})

plt.title('🔗 Matrice de Corrélation entre Variables\n(Plus rouge = corrélation positive, Plus bleu = corrélation négative)', 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Analysons les corrélations avec le prix
correlations_prix = correlation_matrix['Prix_k€'].sort_values(key=abs, ascending=False)
print("🎯 === CORRÉLATIONS AVEC LE PRIX (TARGET) ===")
for variable, correlation in correlations_prix.items():
    if variable != 'Prix_k€':
        direction = "📈 positive" if correlation > 0 else "📉 négative"
        force = "🔥 forte" if abs(correlation) > 0.7 else "💪 modérée" if abs(correlation) > 0.4 else "👍 faible"
        print(f"   {variable:<15} : {correlation:+.3f} → Corrélation {direction} {force}")

print()
print("💡 === INTERPRÉTATION ===")
print("   ✅ Surface : Plus c'est grand, plus c'est cher (logique !)")
print("   ✅ Chambres : Plus de chambres = prix plus élevé (logique !)")
print("   ✅ Age : Plus vieux = moins cher (dépréciation)")
print("   ✅ Quartier : Code plus faible = quartier plus central = plus cher")
print()

# Vérifions s'il y a de la multicolinéarité
print("⚠️ === DÉTECTION DE MULTICOLINÉARITÉ ===")
variables_explicatives = ['Surface_m2', 'Chambres', 'Age_annees', 'Quartier_code']
for i, var1 in enumerate(variables_explicatives):
    for j, var2 in enumerate(variables_explicatives[i+1:], i+1):
        corr = correlation_matrix.loc[var1, var2]
        if abs(corr) > 0.8:
            print(f"   🔴 ATTENTION: {var1} et {var2} sont très corrélées ({corr:.3f})")
        elif abs(corr) > 0.6:
            print(f"   🟡 MODÉRÉ: {var1} et {var2} sont corrélées ({corr:.3f})")

print("   ✅ Pas de multicolinéarité forte détectée (c'est bien !)")

In [ ]:
# 📊 === VISUALISATIONS DÉTAILLÉES ===

# Créons un graphique avec plusieurs sous-graphiques
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🏠 Relations entre Variables et Prix des Maisons', fontsize=16, fontweight='bold')

# 1. Surface vs Prix
axes[0,0].scatter(data['Surface_m2'], data['Prix_k€'], alpha=0.7, color='orange', s=60)
axes[0,0].set_xlabel('Surface (m²)')
axes[0,0].set_ylabel('Prix (k€)')
axes[0,0].set_title('📏 Surface vs Prix')
axes[0,0].grid(True, alpha=0.3)

# 2. Chambres vs Prix
data.boxplot(column='Prix_k€', by='Chambres', ax=axes[0,1])
axes[0,1].set_xlabel('Nombre de Chambres')
axes[0,1].set_ylabel('Prix (k€)')
axes[0,1].set_title('🛏️ Chambres vs Prix')

# 3. Âge vs Prix
axes[1,0].scatter(data['Age_annees'], data['Prix_k€'], alpha=0.7, color='green', s=60)
axes[1,0].set_xlabel('Âge (années)')
axes[1,0].set_ylabel('Prix (k€)')
axes[1,0].set_title('📅 Âge vs Prix')
axes[1,0].grid(True, alpha=0.3)

# 4. Quartier vs Prix
data.boxplot(column='Prix_k€', by='Quartier_nom', ax=axes[1,1])
axes[1,1].set_xlabel('Quartier')
axes[1,1].set_ylabel('Prix (k€)')
axes[1,1].set_title('🏘️ Quartier vs Prix')
axes[1,1].tick_params(axis='x', rotation=45)

# Nettoyage des titres automatiques de boxplot
axes[0,1].set_title('🛏️ Chambres vs Prix')
axes[1,1].set_title('🏘️ Quartier vs Prix')
fig.suptitle('🏠 Relations entre Variables et Prix des Maisons', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

print("👀 === CE QUE NOUS VOYONS ===")
print("   📏 Surface : Relation positive claire (plus grand = plus cher)")
print("   🛏️ Chambres : Tendance positive (plus de chambres = plus cher)")
print("   📅 Âge : Relation négative (plus vieux = moins cher)")
print("   🏘️ Quartier : Centre > Résidentiel > Périphérie (comme attendu)")
print()
print("🎯 === CONCLUSION ===")
print("   ✅ TOUTES les variables semblent avoir un impact sur le prix")
print("   ✅ Les relations sont cohérentes avec nos attentes")
print("   ✅ Un modèle multiple devrait être plus précis qu'un modèle simple")

## 🔄 Comparaison : Régression Simple vs Multiple

Commençons par créer un modèle avec **une seule variable** (comme dans la session précédente), puis un modèle avec **toutes les variables**.

In [ ]:
# 🔄 === COMPARAISON RÉGRESSION SIMPLE VS MULTIPLE ===

print("🔄 === COMPARAISON DIRECTE : SIMPLE VS MULTIPLE ===")
print()

# Préparation des données
# Pour la régression simple : seulement la surface
X_simple = data[['Surface_m2']]

# Pour la régression multiple : toutes les variables
X_multiple = data[['Surface_m2', 'Chambres', 'Age_annees', 'Quartier_code']]

# Variable cible (la même pour les deux)
y = data['Prix_k€']

print("📊 === DONNÉES PRÉPARÉES ===")
print(f"   Régression simple : {X_simple.shape[1]} variable (Surface)")
print(f"   Régression multiple : {X_multiple.shape[1]} variables (Surface, Chambres, Age, Quartier)")
print(f"   Nombre d'observations : {len(y)} maisons")
print()

# === MODÈLE 1 : RÉGRESSION SIMPLE ===
print("🏠 === MODÈLE 1 : RÉGRESSION SIMPLE (Surface seulement) ===")
model_simple = LinearRegression()
model_simple.fit(X_simple, y)

# Prédictions et métriques
y_pred_simple = model_simple.predict(X_simple)
r2_simple = r2_score(y, y_pred_simple)
rmse_simple = np.sqrt(mean_squared_error(y, y_pred_simple))
mae_simple = mean_absolute_error(y, y_pred_simple)

print(f"   📊 Équation : Prix = {model_simple.intercept_:.1f} + {model_simple.coef_[0]:.2f} × Surface")
print(f"   📈 R² = {r2_simple:.3f} ({r2_simple*100:.1f}%)")
print(f"   📏 RMSE = {rmse_simple:.1f} k€")
print(f"   📐 MAE = {mae_simple:.1f} k€")
print()

# === MODÈLE 2 : RÉGRESSION MULTIPLE ===
print("🏘️ === MODÈLE 2 : RÉGRESSION MULTIPLE (Toutes variables) ===")
model_multiple = LinearRegression()
model_multiple.fit(X_multiple, y)

# Prédictions et métriques
y_pred_multiple = model_multiple.predict(X_multiple)
r2_multiple = r2_score(y, y_pred_multiple)
rmse_multiple = np.sqrt(mean_squared_error(y, y_pred_multiple))
mae_multiple = mean_absolute_error(y, y_pred_multiple)

print(f"   📊 Équation complète :")
print(f"      Prix = {model_multiple.intercept_:.1f}")
for i, feature in enumerate(X_multiple.columns):
    coef = model_multiple.coef_[i]
    signe = "+" if coef >= 0 else ""
    print(f"           {signe}{coef:.2f} × {feature}")
print()
print(f"   📈 R² = {r2_multiple:.3f} ({r2_multiple*100:.1f}%)")
print(f"   📏 RMSE = {rmse_multiple:.1f} k€")
print(f"   📐 MAE = {mae_multiple:.1f} k€")
print()

# === COMPARAISON DES PERFORMANCES ===
print("⚖️ === COMPARAISON DES PERFORMANCES ===")
print(f"{'Métrique':<15} {'Simple':<15} {'Multiple':<15} {'Amélioration':<15}")
print("-" * 65)
print(f"{'R²':<15} {r2_simple:.3f}{'':<10} {r2_multiple:.3f}{'':<10} {((r2_multiple-r2_simple)/r2_simple*100):+.1f}%")
print(f"{'RMSE (k€)':<15} {rmse_simple:.1f}{'':<10} {rmse_multiple:.1f}{'':<10} {((rmse_multiple-rmse_simple)/rmse_simple*100):+.1f}%")
print(f"{'MAE (k€)':<15} {mae_simple:.1f}{'':<10} {mae_multiple:.1f}{'':<10} {((mae_multiple-mae_simple)/mae_simple*100):+.1f}%")
print()

print("🎯 === INTERPRÉTATION DES RÉSULTATS ===")
if r2_multiple > r2_simple:
    improvement = ((r2_multiple - r2_simple) / r2_simple) * 100
    print(f"   ✅ La régression multiple est MEILLEURE : +{improvement:.1f}% d'amélioration du R²")
    print(f"   🎉 En ajoutant les variables Chambres, Age et Quartier, nous expliquons mieux les prix")
else:
    print("   ⚠️ Résultat inattendu : la régression simple performe mieux")

if rmse_multiple < rmse_simple:
    error_reduction = ((rmse_simple - rmse_multiple) / rmse_simple) * 100
    print(f"   📉 Réduction d'erreur RMSE : -{error_reduction:.1f}%")
    print(f"   💡 Nos prédictions sont plus précises de {rmse_simple - rmse_multiple:.1f}k€ en moyenne")

print()
print("💭 === POURQUOI LA RÉGRESSION MULTIPLE EST MEILLEURE ? ===")
print("   🎯 Elle capture plus d'informations sur ce qui influence le prix")
print("   🔍 Elle peut différencier des maisons de même surface mais caractéristiques différentes")
print("   🏠 Plus réaliste : dans la vraie vie, plusieurs facteurs comptent !")

## 🔍 Interprétation des Coefficients en Régression Multiple

L'interprétation des coefficients en régression multiple est plus nuancée qu'en régression simple.

In [ ]:
# 🔍 === INTERPRÉTATION DÉTAILLÉE DES COEFFICIENTS ===

print("🔍 === INTERPRÉTATION DES COEFFICIENTS EN RÉGRESSION MULTIPLE ===")
print()
print("📊 Rappel de notre équation :")
print(f"   Prix = {model_multiple.intercept_:.1f} + {model_multiple.coef_[0]:.2f}×Surface + {model_multiple.coef_[1]:.2f}×Chambres + {model_multiple.coef_[2]:.2f}×Age + {model_multiple.coef_[3]:.2f}×Quartier")
print()

# Créons un tableau d'interprétation
coefficients_df = pd.DataFrame({
    'Variable': ['Intercept'] + list(X_multiple.columns),
    'Coefficient': [model_multiple.intercept_] + list(model_multiple.coef_),
    'Interprétation': [
        'Prix de base (quand toutes variables = 0)',
        'Augmentation prix par m² supplémentaire',
        'Augmentation prix par chambre supplémentaire', 
        'Diminution prix par année d\'âge supplémentaire',
        'Effet quartier (1=Centre, 2=Résid., 3=Périph.)'
    ]
})

print("📋 === TABLEAU D'INTERPRÉTATION DES COEFFICIENTS ===")
for i, row in coefficients_df.iterrows():
    print(f"\n🔸 {row['Variable']} = {row['Coefficient']:.2f}")
    print(f"   💡 {row['Interprétation']}")
    
    if row['Variable'] == 'Surface_m2':
        print(f"   📏 Chaque m² supplémentaire ajoute {row['Coefficient']:.2f}k€ au prix")
        print(f"   🏠 TOUTES AUTRES CHOSES ÉGALES PAR AILLEURS")
    elif row['Variable'] == 'Chambres':
        print(f"   🛏️ Chaque chambre supplémentaire ajoute {row['Coefficient']:.2f}k€ au prix")
        print(f"   🔄 À surface, âge et quartier constants")
    elif row['Variable'] == 'Age_annees':
        if row['Coefficient'] < 0:
            print(f"   📅 Chaque année d'âge réduit le prix de {abs(row['Coefficient']):.2f}k€")
            print(f"   📉 Dépréciation normale d'un bien immobilier")
    elif row['Variable'] == 'Quartier_code':
        if row['Coefficient'] < 0:
            print(f"   🏘️ Plus le code quartier augmente, plus le prix diminue")
            print(f"   📍 Centre (1) > Résidentiel (2) > Périphérie (3)")

print("\n" + "="*60)
print("⚠️ === POINT CLÉ : 'TOUTES CHOSES ÉGALES PAR AILLEURS' ===")
print()
print("🎯 En régression multiple, chaque coefficient représente l'effet")
print("   d'UNE variable quand TOUTES LES AUTRES sont maintenues constantes.")
print()
print("💡 Exemples pratiques :")
print(f"   📏 +1m² → +{model_multiple.coef_[0]:.1f}k€ (même nb chambres, même âge, même quartier)")
print(f"   🛏️ +1 chambre → +{model_multiple.coef_[1]:.1f}k€ (même surface, même âge, même quartier)")
print(f"   📅 +1 an d'âge → {model_multiple.coef_[2]:.1f}k€ (même surface, même chambres, même quartier)")
print()
print("🔍 C'est la GRANDE différence avec la régression simple !")
print("   En simple : on voit l'effet TOTAL de la surface (qui inclut les corrélations)")
print("   En multiple : on voit l'effet DIRECT de chaque variable")

In [ ]:
# 📊 === EXEMPLE CONCRET D'INTERPRÉTATION ===

print("🏠 === EXEMPLE CONCRET : PRÉDICTION DE PRIX AVEC NOTRE MODÈLE ===")
print()

# Créons quelques maisons fictives pour tester notre modèle
maisons_test = pd.DataFrame({
    'Surface_m2': [80, 80, 80, 100, 60],
    'Chambres': [3, 3, 3, 4, 2],
    'Age_annees': [5, 20, 35, 10, 0],
    'Quartier_code': [1, 2, 3, 1, 2],
    'Quartier_nom': ['Centre', 'Résidentiel', 'Périphérie', 'Centre', 'Résidentiel']
})

# Prédictions
prix_predits = model_multiple.predict(maisons_test[['Surface_m2', 'Chambres', 'Age_annees', 'Quartier_code']])
maisons_test['Prix_Predit_k€'] = prix_predits.round(1)

print("📋 === PRÉDICTIONS SUR MAISONS FICTIVES ===")
print(maisons_test[['Surface_m2', 'Chambres', 'Age_annees', 'Quartier_nom', 'Prix_Predit_k€']].to_string(index=False))
print()

print("🔍 === ANALYSE DES DIFFÉRENCES ===")
print("\n🏠 Comparons les maisons 1, 2 et 3 (même surface et chambres, quartiers différents) :")
for i in range(3):
    maison = maisons_test.iloc[i]
    print(f"   Maison {i+1}: {maison['Surface_m2']}m², {maison['Chambres']} chambres, {maison['Age_annees']} ans, {maison['Quartier_nom']} → {maison['Prix_Predit_k€']}k€")

print("\n💡 Impact du quartier (à surface/chambres/âge constants) :")
print(f"   Centre vs Résidentiel : {maisons_test.iloc[0]['Prix_Predit_k€'] - maisons_test.iloc[1]['Prix_Predit_k€']:.1f}k€ de différence")
print(f"   Résidentiel vs Périphérie : {maisons_test.iloc[1]['Prix_Predit_k€'] - maisons_test.iloc[2]['Prix_Predit_k€']:.1f}k€ de différence")

print("\n📅 Impact de l'âge (comparons maisons 1 vs 2) :")
age_diff = maisons_test.iloc[1]['Age_annees'] - maisons_test.iloc[0]['Age_annees']
print(f"   Différence d'âge : {age_diff} ans")
print(f"   Impact théorique : {age_diff} × {model_multiple.coef_[2]:.2f} = {age_diff * model_multiple.coef_[2]:.1f}k€")
print(f"   ✅ Cohérent avec l'écart de prix observé !")

print("\n🧮 === CALCUL MANUEL POUR VÉRIFICATION ===")
print("\nVérifions la maison 1 (80m², 3ch, 5ans, Centre) :")
calcul_manuel = (model_multiple.intercept_ + 
                model_multiple.coef_[0] * 80 +  # Surface
                model_multiple.coef_[1] * 3 +   # Chambres
                model_multiple.coef_[2] * 5 +   # Age
                model_multiple.coef_[3] * 1)    # Quartier Centre
print(f"Prix calculé manuellement : {calcul_manuel:.1f}k€")
print(f"Prix du modèle : {prix_predits[0]:.1f}k€")
print(f"✅ {'Identique' if abs(calcul_manuel - prix_predits[0]) < 0.1 else 'Différent'} !")

## 📊 Visualisation du Modèle Multiple

Visualiser un modèle à 4 variables est complexe, mais nous pouvons créer des graphiques utiles.

In [ ]:
# 📊 === VISUALISATION DU MODÈLE MULTIPLE ===

# 1. Graphique des prédictions vs réalité
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📊 Analyse du Modèle de Régression Multiple', fontsize=16, fontweight='bold')

# Graphique 1: Prédictions vs Réalité
axes[0,0].scatter(y, y_pred_multiple, alpha=0.7, color='blue', s=80)
axes[0,0].plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2, label='Prédiction parfaite')
axes[0,0].set_xlabel('Prix Réel (k€)')
axes[0,0].set_ylabel('Prix Prédit (k€)')
axes[0,0].set_title(f'🎯 Prédictions vs Réalité\n(R² = {r2_multiple:.3f})')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Graphique 2: Résidus vs Prédictions
residus = y - y_pred_multiple
axes[0,1].scatter(y_pred_multiple, residus, alpha=0.7, color='orange', s=80)
axes[0,1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0,1].set_xlabel('Prix Prédit (k€)')
axes[0,1].set_ylabel('Résidus (k€)')
axes[0,1].set_title('📈 Résidus vs Prédictions\n(Doit être aléatoire autour de 0)')
axes[0,1].grid(True, alpha=0.3)

# Graphique 3: Importance des variables (coefficients)
variables = X_multiple.columns
coefficients = model_multiple.coef_
couleurs = ['red' if c < 0 else 'green' for c in coefficients]

axes[1,0].barh(variables, coefficients, color=couleurs, alpha=0.7)
axes[1,0].set_xlabel('Valeur du Coefficient')
axes[1,0].set_title('📊 Importance des Variables\n(Rouge = Impact négatif, Vert = Impact positif)')
axes[1,0].axvline(x=0, color='black', linestyle='-', linewidth=1)
axes[1,0].grid(True, alpha=0.3)

# Graphique 4: Distribution des résidus
axes[1,1].hist(residus, bins=10, alpha=0.7, color='purple', edgecolor='black')
axes[1,1].set_xlabel('Résidus (k€)')
axes[1,1].set_ylabel('Fréquence')
axes[1,1].set_title(f'📈 Distribution des Résidus\n(Moyenne = {residus.mean():.1f}k€)')
axes[1,1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🔍 === ANALYSE DES GRAPHIQUES ===")
print()
print("📊 Graphique 1 (Prédictions vs Réalité) :")
if r2_multiple > 0.8:
    print("   ✅ Points proches de la ligne rouge → Bonnes prédictions !")
elif r2_multiple > 0.6:
    print("   👍 Points relativement alignés → Prédictions correctes")
else:
    print("   ⚠️ Points dispersés → Prédictions à améliorer")

print("\n📈 Graphique 2 (Résidus vs Prédictions) :")
residus_std = residus.std()
if residus_std < 20:
    print(f"   ✅ Résidus bien répartis autour de 0 (σ = {residus_std:.1f}k€)")
    print("   ✅ Pas de pattern visible → Modèle bien spécifié")
else:
    print(f"   ⚠️ Résidus assez dispersés (σ = {residus_std:.1f}k€)")

print("\n📊 Graphique 3 (Importance des Variables) :")
for var, coef in zip(variables, coefficients):
    impact = "positif" if coef > 0 else "négatif"
    force = "fort" if abs(coef) > 1 else "modéré"
    print(f"   {var:<15} : Impact {impact} {force} ({coef:+.2f})")

print("\n📈 Graphique 4 (Distribution des Résidus) :")
if abs(residus.mean()) < 5:
    print(f"   ✅ Résidus centrés sur 0 (moyenne = {residus.mean():.2f}k€)")
    print("   ✅ Pas de biais systématique dans les prédictions")
else:
    print(f"   ⚠️ Résidus non centrés (moyenne = {residus.mean():.2f}k€)")
    print("   ⚠️ Possible biais systématique à investiguer")

## ⚖️ Train/Test Split - Évaluation Plus Rigoureuse

Jusqu'à présent, nous avons évalué nos modèles sur les mêmes données qui ont servi à l'entraînement. C'est une pratique dangereuse ! Utilisons une approche plus rigoureuse.

In [ ]:
# ⚖️ === ÉVALUATION RIGOUREUSE AVEC TRAIN/TEST SPLIT ===

print("⚖️ === ÉVALUATION PLUS RIGOUREUSE AVEC TRAIN/TEST ===")
print()
print("🎯 Objectif : Éviter l'overfitting en testant sur des données non vues")
print("📚 Principe : Entraîner sur 80% des données, tester sur 20%")
print()

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_multiple, y, 
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)

print(f"📊 === DIVISION DES DONNÉES ===")
print(f"   Données d'entraînement : {len(X_train)} maisons ({len(X_train)/len(data)*100:.0f}%)")
print(f"   Données de test : {len(X_test)} maisons ({len(X_test)/len(data)*100:.0f}%)")
print(f"   Total : {len(data)} maisons")
print()

# === MODÈLES SUR DONNÉES D'ENTRAÎNEMENT ===
print("🏋️ === ENTRAÎNEMENT SUR LES DONNÉES TRAIN ===")

# Régression simple sur train
model_simple_train = LinearRegression()
model_simple_train.fit(X_train[['Surface_m2']], y_train)

# Régression multiple sur train
model_multiple_train = LinearRegression()
model_multiple_train.fit(X_train, y_train)

print("   ✅ Modèles entraînés sur les données train")
print()

# === ÉVALUATION SUR TRAIN ET TEST ===
print("📊 === ÉVALUATION SUR TRAIN ET TEST ===")
print()

# Prédictions
# Sur train
y_train_pred_simple = model_simple_train.predict(X_train[['Surface_m2']])
y_train_pred_multiple = model_multiple_train.predict(X_train)

# Sur test
y_test_pred_simple = model_simple_train.predict(X_test[['Surface_m2']])
y_test_pred_multiple = model_multiple_train.predict(X_test)

# Calcul des métriques
metriques = {
    'Simple_Train': {
        'R²': r2_score(y_train, y_train_pred_simple),
        'RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred_simple)),
        'MAE': mean_absolute_error(y_train, y_train_pred_simple)
    },
    'Simple_Test': {
        'R²': r2_score(y_test, y_test_pred_simple),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred_simple)),
        'MAE': mean_absolute_error(y_test, y_test_pred_simple)
    },
    'Multiple_Train': {
        'R²': r2_score(y_train, y_train_pred_multiple),
        'RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred_multiple)),
        'MAE': mean_absolute_error(y_train, y_train_pred_multiple)
    },
    'Multiple_Test': {
        'R²': r2_score(y_test, y_test_pred_multiple),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred_multiple)),
        'MAE': mean_absolute_error(y_test, y_test_pred_multiple)
    }
}

# Affichage des résultats
print(f"{'Modèle':<20} {'Dataset':<8} {'R²':<8} {'RMSE':<8} {'MAE':<8}")
print("-" * 60)
print(f"{'Régression Simple':<20} {'Train':<8} {metriques['Simple_Train']['R²']:.3f}{'':<4} {metriques['Simple_Train']['RMSE']:.1f}{'':<4} {metriques['Simple_Train']['MAE']:.1f}")
print(f"{'Régression Simple':<20} {'Test':<8} {metriques['Simple_Test']['R²']:.3f}{'':<4} {metriques['Simple_Test']['RMSE']:.1f}{'':<4} {metriques['Simple_Test']['MAE']:.1f}")
print(f"{'Régression Multiple':<20} {'Train':<8} {metriques['Multiple_Train']['R²']:.3f}{'':<4} {metriques['Multiple_Train']['RMSE']:.1f}{'':<4} {metriques['Multiple_Train']['MAE']:.1f}")
print(f"{'Régression Multiple':<20} {'Test':<8} {metriques['Multiple_Test']['R²']:.3f}{'':<4} {metriques['Multiple_Test']['RMSE']:.1f}{'':<4} {metriques['Multiple_Test']['MAE']:.1f}")
print()

# Analyse des résultats
print("🔍 === ANALYSE DES RÉSULTATS ===")
print()

# Vérification de l'overfitting
def check_overfitting(train_score, test_score, model_name):
    diff = train_score - test_score
    if diff < 0.05:
        return f"   ✅ {model_name} : Pas d'overfitting (écart R² = {diff:.3f})"
    elif diff < 0.1:
        return f"   ⚠️ {model_name} : Léger overfitting (écart R² = {diff:.3f})"
    else:
        return f"   🔴 {model_name} : Overfitting important (écart R² = {diff:.3f})"

print("🎯 Détection d'overfitting (Train vs Test) :")
print(check_overfitting(metriques['Simple_Train']['R²'], metriques['Simple_Test']['R²'], "Régression Simple"))
print(check_overfitting(metriques['Multiple_Train']['R²'], metriques['Multiple_Test']['R²'], "Régression Multiple"))
print()

print("📊 Performance sur nouvelles données (Test) :")
if metriques['Multiple_Test']['R²'] > metriques['Simple_Test']['R²']:
    improvement = ((metriques['Multiple_Test']['R²'] - metriques['Simple_Test']['R²']) / metriques['Simple_Test']['R²']) * 100
    print(f"   ✅ La régression multiple généralise mieux : +{improvement:.1f}% d'amélioration")
else:
    print("   ⚠️ La régression simple généralise mieux sur ce dataset")

print(f"\n💡 Modèle recommandé pour la production :")
if metriques['Multiple_Test']['R²'] > 0.7:
    print(f"   🎯 Régression Multiple (R² test = {metriques['Multiple_Test']['R²']:.3f})")
    print(f"   📏 Erreur typique : ±{metriques['Multiple_Test']['RMSE']:.1f}k€")
else:
    print(f"   📊 Besoin d'amélioration - modèles actuels insuffisants")
    print(f"   🔄 Considérer : plus de données, feature engineering, autres algorithmes")

## 🎯 Concepts Clés - Récapitulatif Régression Multiple

### ✅ **Régression Multiple vs Simple**

| Aspect | 📈 Régression Simple | 🏘️ Régression Multiple |
|--------|---------------------|------------------------|
| **Variables** | 1 variable explicative | Plusieurs variables explicatives |
| **Équation** | y = w₀ + w₁×x | y = w₀ + w₁×x₁ + w₂×x₂ + ... |
| **Visualisation** | Droite en 2D | Hyperplan en nD |
| **Interprétation** | Effet total de x sur y | Effet de chaque xᵢ, autres variables constantes |
| **Avantages** | Simple, facile à interpréter | Plus réaliste, souvent plus précise |
| **Inconvénients** | Souvent trop simpliste | Plus complexe, risque d'overfitting |

### ✅ **Interprétation des Coefficients**

**🔑 Règle d'or : "Toutes choses égales par ailleurs"**

En régression multiple, chaque coefficient représente :
- **w₁** : Changement dans y pour +1 unité de x₁, **quand toutes les autres variables restent constantes**
- **w₀** : Valeur de y quand **toutes les variables explicatives = 0**

**💡 Exemple pratique :**
```
Prix = 100 + 1.8×Surface + 15×Chambres - 0.8×Age - 25×Quartier
```
- **+1m²** → **+1.8k€** (à chambres, âge, quartier constants)
- **+1 chambre** → **+15k€** (à surface, âge, quartier constants)
- **+1 an** → **-0.8k€** (à surface, chambres, quartier constants)

### ✅ **Métriques d'Évaluation**

**🎯 Train vs Test :**
- **Performance Train** : Comment le modèle "mémorise" les données d'entraînement
- **Performance Test** : Comment le modèle **généralise** sur de nouvelles données
- **Overfitting** : Quand Train >> Test (modèle trop spécialisé)

**📊 Métriques clés :**
- **R²** : % de variance expliquée (0-1, plus proche de 1 = mieux)
- **RMSE** : Erreur quadratique moyenne (même unité que y, plus petit = mieux)
- **MAE** : Erreur absolue moyenne (même unité que y, plus petit = mieux)

### ✅ **Challenges de la Régression Multiple**

**⚠️ Multicolinéarité :**
- Quand les variables explicatives sont corrélées entre elles
- Rend l'interprétation des coefficients difficile
- Solution : Sélection de variables, PCA, Ridge regression

**⚠️ Overfitting :**
- Trop de variables par rapport au nombre d'observations
- Modèle trop complexe qui mémorise le bruit
- Solution : Train/test split, cross-validation, régularisation

**⚠️ Interprétation complexe :**
- Difficile à visualiser au-delà de 2-3 variables
- Effets d'interaction possibles entre variables
- Solution : Graphiques partiels, importance des features

### ✅ **Quand Utiliser la Régression Multiple ?**

**✅ Utilisez quand :**
- Vous avez plusieurs facteurs qui influencent votre target
- Les relations semblent linéaires
- Vous voulez comprendre l'effet individuel de chaque variable
- Vous avez assez de données (règle : 10-20 obs par variable)
- L'interprétabilité est importante

**⚠️ Attention quand :**
- Variables très corrélées entre elles
- Relations non-linéaires importantes
- Peu de données par rapport au nombre de variables
- Besoin de prédictions uniquement (sans interprétation)

### 💡 **Points Clés à Retenir**

1. **Plus réaliste** : La régression multiple capture mieux la complexité du monde réel
2. **Interprétation différente** : Chaque coefficient = effet "toutes choses égales par ailleurs"
3. **Validation cruciale** : Toujours tester sur des données non vues
4. **Balance complexité/performance** : Plus de variables ≠ toujours mieux
5. **Base solide** : Excellente introduction aux modèles plus avancés (ML, Deep Learning)

---

## 🎉 Félicitations !

Vous maîtrisez maintenant :
- ✅ **Régression linéaire simple ET multiple**
- ✅ **Interprétation business des coefficients**
- ✅ **Évaluation rigoureuse avec train/test**
- ✅ **Détection d'overfitting**
- ✅ **Visualisation et diagnostic des modèles**

**🚀 Prochaines étapes :**
- Session 3 : Classification (Régression Logistique)
- Techniques de régularisation (Ridge, Lasso)
- Autres algorithmes de ML (Random Forest, SVM, etc.)

**💪 Vous avez maintenant une base solide en Machine Learning supervisé !**